In [30]:
import pandas as pd
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import matplotlib.pyplot as plt
import datetime
import numpy as np
from matplotlib import pyplot
from math import sqrt
from numpy import concatenate
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
import xgboost as xgb
from __future__ import division 
#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [31]:
#!pip install openpyxl

In [32]:
# Importing Data
df_people = pd.read_csv("full occ csv (1).csv")
df_weather = pd.read_csv("weather-1 (1).csv")
df_electricity = pd.read_excel("consumption.xlsx")

In [33]:
#Checking NA and duplicates
df_people = df_people.dropna(how='all')
df_people =  df_people.drop_duplicates()

In [34]:
#Checking NA and duplicates
df_weather = df_weather.dropna(how='all')
df_weather = df_weather.drop_duplicates()

In [35]:
#conveting Date column into Date time Format
df_people['Date'] = pd.to_datetime(df_people['Date'], format='%d/%m/%Y')
df_weather['READING DATE'] = pd.to_datetime(df_weather['READING DATE'], format='%Y-%m-%d')

In [36]:
#Getting only same date entries from whether columns, becuase It has lowest date entries
df_people = df_people[df_people['Date'].isin(list(df_weather['READING DATE']))].reset_index().drop(['index'],1)
df_electricity = df_electricity[df_electricity['READING DATE'].isin(list(df_weather['READING DATE']))].reset_index().drop(['index'],1)

In [37]:
#removing unwanted columns
df_electricity = df_electricity.drop(["H","OPTIMA Half Hourly DATA","Unnamed: 2","CHANNEL TYPE","UNITS"],axis = 1)

In [38]:
#removing unwanted columns
electricity_drop_columns = [list(df_electricity.columns)[i] for i in range(len(list(df_electricity.columns))) if i%2!=0]
df_electricity = df_electricity.drop(electricity_drop_columns,1)
df_electricity['00:00'] = df_electricity[list(df_electricity.columns)[-1]]
df_electricity = df_electricity.drop([list(df_electricity.columns)[-2]],1)
df_electricity['Date'] = df_electricity['READING DATE']
df_electricity = df_electricity.drop(['READING DATE'],1)

In [39]:
#removing unwanted columns
whether_drop_columns = [list(df_weather.columns)[i] for i in range(len(list(df_weather.columns))) if i%2!=0]
df_weather = df_weather.drop(whether_drop_columns,1)
df_weather['00:00'] = df_weather['23:59:00']
df_weather = df_weather.drop(['23:59:00'],1)
df_weather['Date'] = df_weather['READING DATE']
df_weather = df_weather.drop(['READING DATE'],1)

In [40]:
for i,j in enumerate(df_electricity["11:00"]):
    if j == 1272.5:
        df_electricity["11:00"][i] = 0

In [41]:
#replacing bad entries from df_people
for i,j in enumerate(df_people["13:00"]):
    if j == 'UID card access problems':
        df_people["13:00"][i] = np.nan

for k in df_people.columns:
    for i,j in enumerate(df_people[k]):
        if j == 'Sentry Juno installation - Sentry offline, no data 9th -22nd July' or j == 'Sentry Juno live but no data yet' or j == "Sentry Juno put into offline mode" or j == "#" or j == " ":
            df_people[k][i] = np.nan
            


In [42]:
start_date = '2018-10-01'
end_date =  '2019-7-01'

In [43]:
df_people =  df_people[(df_people['Date'] >= start_date) & (df_people['Date'] <= end_date)].reset_index().drop(['index'],1)
df_weather =  df_weather[(df_weather['Date'] >= start_date) & (df_weather['Date'] <= end_date)].reset_index().drop(['index'],1)
df_electricity =  df_electricity[(df_electricity['Date'] >= start_date) & (df_electricity['Date'] <= end_date)].reset_index().drop(['index'],1)

In [45]:
#filling na with 0
df_people = df_people.fillna(0)

In [46]:
#filling na with 0
df_weather = df_weather.fillna(0)

In [47]:
df_people.shape,df_weather.shape,df_electricity.shape

((274, 25), (274, 25), (274, 25))

In [48]:
def get_predicted_x_df(df):
    #creating empty dataframe
    temp_df = pd.DataFrame()

    #running for loop on two columns people and whether to run LSTM on them to predict future values
    for cols in df.drop(['Date'], axis = 1).columns:
        timeseries_data = df[cols]
        scaler = MinMaxScaler()
        print(f'Cols : {cols}')
        scaler.fit(np.array(timeseries_data).reshape(-1, 1))
        scaled_train = scaler.transform(np.array(timeseries_data).reshape(-1, 1))
        n_input = 30
        n_features = 1
        generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size = 64)
        X, y = generator[0]
        model = Sequential()
        model.add(LSTM(64, activation='relu', input_shape = (X.shape[1], X.shape[2]),return_sequences= True))
        model.add(LSTM(32, activation='relu', return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(32, activation='relu', return_sequences=False))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        model.compile(optimizer = 'adam', loss = 'mse')
        print(f'Training for columns : {cols}')
        model.fit(generator, epochs = 5, verbose = 1)
        test_predictions = []
        first_eval_batch = scaled_train[-n_input:]
        current_batch = first_eval_batch.reshape((1, n_input, n_features))
        future = 30 # this will predict no. of future values.
        for i in range(future):
            current_pred = model.predict(current_batch)[0]
            test_predictions.append(current_pred)
            current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

        true_predictions = scaler.inverse_transform(test_predictions)
        temp_df[cols] = true_predictions.flatten()
    return temp_df

def compute_r2(y_true, y_predicted):
    sse = sum((y_true - y_predicted)**2)
    tse = (len(y_true) - 1) * np.var(y_true, ddof=1)
    r2_score = 1 - (sse / tse)
    return r2_score

In [49]:
predicted_df_x_people = get_predicted_x_df(df_people)
predicted_df_x_weather = get_predicted_x_df(df_weather)

Cols : 00:00
Training for columns : 00:00
Epoch 1/5
4/4 [==============================] - 0s 120ms/step - loss: 0.1451
Epoch 2/5
4/4 [==============================] - 0s 108ms/step - loss: 0.1178
Epoch 3/5
4/4 [==============================] - 0s 104ms/step - loss: 0.0915
Epoch 4/5
4/4 [==============================] - 1s 134ms/step - loss: 0.0519
Epoch 5/5
4/4 [==============================] - 0s 97ms/step - loss: 0.0605
Cols : 01:00
Training for columns : 01:00
Epoch 1/5
4/4 [==============================] - 1s 132ms/step - loss: 0.1185
Epoch 2/5
4/4 [==============================] - 0s 117ms/step - loss: 0.1012
Epoch 3/5
4/4 [==============================] - 1s 155ms/step - loss: 0.0881
Epoch 4/5
4/4 [==============================] - 0s 117ms/step - loss: 0.0639
Epoch 5/5
4/4 [==============================] - 0s 96ms/step - loss: 0.0432
Cols : 02:00
Training for columns : 02:00
Epoch 1/5
4/4 [==============================] - 1s 130ms/step - loss: 0.0806
Epoch 2/5
4/4 [===

4/4 [==============================] - 0s 108ms/step - loss: 0.0439
Epoch 3/5
4/4 [==============================] - 0s 113ms/step - loss: 0.0389
Epoch 4/5
4/4 [==============================] - 1s 145ms/step - loss: 0.0315
Epoch 5/5
4/4 [==============================] - 1s 171ms/step - loss: 0.0273
Cols : 09:00
Training for columns : 09:00
Epoch 1/5
4/4 [==============================] - 1s 141ms/step - loss: 0.0669
Epoch 2/5
4/4 [==============================] - 1s 190ms/step - loss: 0.0567
Epoch 3/5
4/4 [==============================] - 1s 147ms/step - loss: 0.0520
Epoch 4/5
4/4 [==============================] - 0s 117ms/step - loss: 0.0442
Epoch 5/5
4/4 [==============================] - 0s 107ms/step - loss: 0.0382
Cols : 10:00
Training for columns : 10:00
Epoch 1/5
4/4 [==============================] - 1s 130ms/step - loss: 0.1280
Epoch 2/5
4/4 [==============================] - 1s 134ms/step - loss: 0.1138
Epoch 3/5
4/4 [==============================] - 1s 154ms/step - los

Training for columns : 17:00
Epoch 1/5
4/4 [==============================] - 1s 186ms/step - loss: 0.4487
Epoch 2/5
4/4 [==============================] - 0s 125ms/step - loss: 0.4155
Epoch 3/5
4/4 [==============================] - 1s 172ms/step - loss: 0.3643
Epoch 4/5
4/4 [==============================] - 1s 125ms/step - loss: 0.3073
Epoch 5/5
4/4 [==============================] - 1s 145ms/step - loss: 0.2262
Cols : 18:00
Training for columns : 18:00
Epoch 1/5
4/4 [==============================] - 1s 126ms/step - loss: 0.3410
Epoch 2/5
4/4 [==============================] - 1s 155ms/step - loss: 0.2668
Epoch 3/5
4/4 [==============================] - 1s 143ms/step - loss: 0.1915
Epoch 4/5
4/4 [==============================] - 0s 108ms/step - loss: 0.0798
Epoch 5/5
4/4 [==============================] - 0s 119ms/step - loss: 0.0733
Cols : 19:00
Training for columns : 19:00
Epoch 1/5
4/4 [==============================] - 1s 148ms/step - loss: 0.2707
Epoch 2/5
4/4 [==============

Training for columns : 03:00
Epoch 1/5
4/4 [==============================] - 0s 109ms/step - loss: 0.1772
Epoch 2/5
4/4 [==============================] - 0s 95ms/step - loss: 0.1556
Epoch 3/5
4/4 [==============================] - 0s 96ms/step - loss: 0.1404
Epoch 4/5
4/4 [==============================] - 0s 103ms/step - loss: 0.1141
Epoch 5/5
4/4 [==============================] - 0s 120ms/step - loss: 0.0872
Cols : 04:00
Training for columns : 04:00
Epoch 1/5
4/4 [==============================] - 0s 101ms/step - loss: 0.1385
Epoch 2/5
4/4 [==============================] - 0s 101ms/step - loss: 0.1039
Epoch 3/5
4/4 [==============================] - 0s 101ms/step - loss: 0.0549
Epoch 4/5
4/4 [==============================] - 0s 102ms/step - loss: 0.0407
Epoch 5/5
4/4 [==============================] - 0s 111ms/step - loss: 0.0391
Cols : 05:00
Training for columns : 05:00
Epoch 1/5
4/4 [==============================] - 0s 118ms/step - loss: 0.1748
Epoch 2/5
4/4 [================

4/4 [==============================] - 1s 126ms/step - loss: 0.1072
Epoch 5/5
4/4 [==============================] - 1s 169ms/step - loss: 0.0573
Cols : 12:00
Training for columns : 12:00
Epoch 1/5
4/4 [==============================] - 0s 102ms/step - loss: 0.1611
Epoch 2/5
4/4 [==============================] - 0s 101ms/step - loss: 0.1394
Epoch 3/5
4/4 [==============================] - 0s 98ms/step - loss: 0.1117
Epoch 4/5
4/4 [==============================] - 0s 104ms/step - loss: 0.0674
Epoch 5/5
4/4 [==============================] - 0s 118ms/step - loss: 0.0395
Cols : 13:00
Training for columns : 13:00
Epoch 1/5
4/4 [==============================] - 0s 117ms/step - loss: 0.1675
Epoch 2/5
4/4 [==============================] - 0s 108ms/step - loss: 0.1455
Epoch 3/5
4/4 [==============================] - 0s 105ms/step - loss: 0.1279
Epoch 4/5
4/4 [==============================] - 0s 99ms/step - loss: 0.1039
Epoch 5/5
4/4 [==============================] - 0s 111ms/step - loss:

Training for columns : 20:00
Epoch 1/5
4/4 [==============================] - 0s 114ms/step - loss: 0.1914
Epoch 2/5
4/4 [==============================] - 0s 120ms/step - loss: 0.1683
Epoch 3/5
4/4 [==============================] - 1s 127ms/step - loss: 0.1356
Epoch 4/5
4/4 [==============================] - 1s 130ms/step - loss: 0.0956
Epoch 5/5
4/4 [==============================] - 1s 154ms/step - loss: 0.0541
Cols : 21:00
Training for columns : 21:00
Epoch 1/5
4/4 [==============================] - 1s 134ms/step - loss: 0.2301
Epoch 2/5
4/4 [==============================] - 0s 118ms/step - loss: 0.1825
Epoch 3/5
4/4 [==============================] - 1s 157ms/step - loss: 0.1393
Epoch 4/5
4/4 [==============================] - 1s 166ms/step - loss: 0.0928
Epoch 5/5
4/4 [==============================] - 1s 144ms/step - loss: 0.0449
Cols : 22:00
Training for columns : 22:00
Epoch 1/5
4/4 [==============================] - 1s 140ms/step - loss: 0.2081
Epoch 2/5
4/4 [==============

In [62]:
mse1 = {}
r2 = {}
train_df = pd.DataFrame()
prediction_df = pd.DataFrame()
predicted_df_y_power_consumption = pd.DataFrame()
#getting future date hour values
forecast_period_dates = []
date = datetime.datetime(2021,11,13)
for i in range(30): 
    date += datetime.timedelta(days=1)
    forecast_period_dates.append(date)
predicted_df_y_power_consumption['Dates'] = forecast_period_dates
predicted_df_x_people['Dates'] = forecast_period_dates
predicted_df_x_weather['Dates'] = forecast_period_dates

In [63]:
for i in list(predicted_df_x_people.drop(['Dates'],1).columns):
    print(i)
    train_df['people_'+i]  =  df_people[i].astype(float)
    train_df['weather_'+i]  =  df_weather[i]
    print('people_'+i,'weather_'+i)
    X = train_df[['people_'+i,'weather_'+i]]
    y = df_electricity[i]
    X_train, X_test, y_train, y_test = tts(X,y,test_size=0.25,random_state=42)
    xgbr = xgb.XGBRegressor(seed = 20,n_estimators = 500,max_depth = 3,learning_rate = 0.01,colsample_bytree = 0.7)
    rf1=RandomForestRegressor(n_estimators = 200, min_samples_split = 5,min_samples_leaf = 4,max_features = 'auto',max_depth = 10,bootstrap = True)
    GBR=GradientBoostingRegressor(subsample = 1,n_estimators = 500, max_depth = 1,  learning_rate = 0.01 )
    model = VotingRegressor(estimators=[('gb', GBR), ('rf', rf1),('xgb',xgbr)])
    model = model.fit(X_train,y_train)
    pred = model.predict(X_test)
    mse1[i] = mean_squared_error(pred,y_test)  
    r2[i] = r2_score(y_test,pred)#compute_r2(y_test,pred)
    prediction_df['people_'+i]  =  predicted_df_x_people[i]
    prediction_df['weather_'+i]  =  predicted_df_x_weather[i]
    power_consumption_pred = model.predict(prediction_df[['people_'+i,'weather_'+i]])
    predicted_df_y_power_consumption[i] = power_consumption_pred
    print("done")

00:00
people_00:00 weather_00:00
done
01:00
people_01:00 weather_01:00
done
02:00
people_02:00 weather_02:00
done
03:00
people_03:00 weather_03:00
done
04:00
people_04:00 weather_04:00
done
05:00
people_05:00 weather_05:00
done
06:00
people_06:00 weather_06:00
done
07:00
people_07:00 weather_07:00
done
08:00
people_08:00 weather_08:00
done
09:00
people_09:00 weather_09:00
done
10:00
people_10:00 weather_10:00
done
11:00
people_11:00 weather_11:00
done
12:00
people_12:00 weather_12:00
done
13:00
people_13:00 weather_13:00
done
14:00
people_14:00 weather_14:00
done
15:00
people_15:00 weather_15:00
done
16:00
people_16:00 weather_16:00
done
17:00
people_17:00 weather_17:00
done
18:00
people_18:00 weather_18:00
done
19:00
people_19:00 weather_19:00
done
20:00
people_20:00 weather_20:00
done
21:00
people_21:00 weather_21:00
done
22:00
people_22:00 weather_22:00
done
23:00
people_23:00 weather_23:00
done


In [32]:
predicted_df_y_power_consumption.to_csv('predicted.csv')

In [64]:
print('MSE : ',sum(list(mse1.values()))/len(list(mse1.values())))

MSE :  1.0376553116097178


In [65]:
print('R-Squared : ',sum(list(r2.values()))/len(list(r2.values()))*100)

R-Squared :  7.571513034688829
